# Question Answering

This notebook uses some generic prompts/language models to evaluate a question answering system.

In [1]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores.faiss import FAISS
from langchain.text_splitter import CharacterTextSplitter
from langchain import OpenAI, VectorDBQA

In [2]:
with open('../state_of_the_union.txt') as f:
    state_of_the_union = f.read()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_text(state_of_the_union)

embeddings = OpenAIEmbeddings()
docsearch = FAISS.from_texts(texts, embeddings)

In [3]:
qa = VectorDBQA.from_llm(llm=OpenAI(), vectorstore=docsearch)

In [4]:
examples = [
    {
        "query": "What did the president say about Ketanji Brown Jackson",
        "answer": "He praised her legal ability and said he nominated her for the supreme court."
    },
    {
        "query": "What did the president say about Michael Jackson",
        "answer": "Nothing"
    }
]

In [5]:
qa.run(examples[0]['query'])

" The president said that Ketanji Brown Jackson is one of the nation's top legal minds, a former top litigator in private practice and a former federal public defender, and that she will continue Justice Breyer's legacy of excellence."

In [6]:
qa.run(examples[1]['query'])

' The president did not mention Michael Jackson.'

In [7]:
from langchain.evaluation.qa import QAEvaluator

In [8]:
evaluator = QAEvaluator.from_llm(OpenAI(temperature=0))

In [9]:
predictions, scored_outputs, scores = evaluator.run_and_score(examples, qa)

In [10]:
predictions

[{'query': 'What did the president say about Ketanji Brown Jackson',
  'answer': 'He praised her legal ability and said he nominated her for the supreme court.',
  'result': " The president said that Ketanji Brown Jackson is one of the nation's top legal minds and that she will continue Justice Breyer's legacy of excellence."},
 {'query': 'What did the president say about Michael Jackson',
  'answer': 'Nothing',
  'result': ' The president did not mention Michael Jackson in this context.'}]

In [11]:
scored_outputs

[{'grade': ' CORRECT'}, {'grade': ' CORRECT'}]

In [12]:
scores

Counter({' CORRECT': 2})